In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Scrape from wikipedia

In [78]:
url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_New_York_City'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

In [139]:
df_dict = dict()
df_dict['date'] = list()
df_dict['cases'] = list()
df_dict['deaths'] = list()
df_dict['recovered'] = list()
for tr in trs:
    dt, cases, deaths = tr.find_all('td', attrs={'class':'bb-04em'})
    date = dt.get_text().replace(',', '')
    cases = cases.find('span', attrs={'class':'cbs-ibr'}).get_text().replace(',', '')
    deaths = deaths.find('span', attrs={'class':'cbs-ibr'}).get_text().replace(',', '')

    recovered = tr.find_all('div', attrs={'class': 'bb-fl'})
    if len(recovered) >= 3:
        recovered = recovered[1].get('title').replace(',', '')
    else:
        recovered = '0'   
    df_dict['date'].append(date)
    df_dict['cases'].append(cases)
    df_dict['deaths'].append(deaths)
    df_dict['recovered'].append(recovered)

In [140]:
new_york = pd.DataFrame(df_dict)

In [141]:
new_york['date'] = pd.to_datetime(new_york['date'])
new_york['recovered'] = new_york['recovered'].astype(int)
new_york['cases'] = new_york['cases'].astype(int)
new_york['deaths'] = new_york['deaths'].astype(int)
new_york['active'] = new_york['cases'] - new_york['recovered'] - new_york['deaths']

new_york = new_york.set_index('date')

### Save data

In [143]:
new_york.to_csv('data/master_data/new_york/from_wiki.csv')